In [1]:
import yt
import numpy as np
from abc import ABC

In [2]:
class CL_Analysis(ABC):
    def Theta(self, i, j, k):
        phi_1_Re, phi_1_Im, phi_2_Re, phi_2_Im = self.get_fields(i, j, k)
        return np.arctan((phi_1_Im + phi_2_Re)/(phi_1_Re - phi_2_Im))
    
    def Circulation(self, radius):
        center_i = int(self.domain_dimensions[0]/2)
        center_j = int(self.domain_dimensions[1]/2)
        center_k = int(self.domain_dimensions[2]/2)

        circulation = 0

        i_left = center_i - radius
        i_right = center_i + radius
        j_bottom = center_j - radius
        j_top = center_j + radius

        for j in range(self.domain_dimensions[1]):
            # Left side of loop except top left corner
            if j >= j_bottom and j < j_top:
                tdiff = self.Theta(i_left, j+1, center_k) - self.Theta(i_left, j, center_k)
                print("adding ({}, {}): {}".format(i_left, j, tdiff))
                circulation += tdiff

            # Top left corner of loop
            elif j == j_top:
                tdiff = self.Theta(i_left+1, j, center_k) - self.Theta(i_left, j, center_k)
                print("adding ({}, {}): {}".format(i_left, j, tdiff))
                circulation += tdiff
                
            # Right side of loop except bottom right corner
            if j > j_bottom and j <= j_top:
                tdiff = self.Theta(i_right, j-1, center_k) - self.Theta(i_right, j, center_k)
                print("adding ({}, {}): {}".format(i_right, j, tdiff))
                circulation += tdiff
    
            # Bottom right corner of loop
            elif j == j_bottom:
                tdiff = self.Theta(i_right-1, j, center_k) - self.Theta(i_right, j, center_k)
                print("adding ({}, {}): {}".format(i_right, j, tdiff))
                circulation += tdiff

        for i in range(self.domain_dimensions[0]):
            # Top and Bottom sides of loop except corners
            if i > i_left and i < i_right:
                tdiff = self.Theta(i+1, j_top, center_k) - self.Theta(i, j_top, center_k)
                circulation += tdiff
                print("adding ({}, {}): {}".format(i, j_top, tdiff))

                tdiff = self.Theta(i-1, j_bottom, center_k) - self.Theta(i, j_bottom, center_k)
                circulation += tdiff
                print("adding ({}, {}): {}".format(i, j_bottom, tdiff))

        circulation = circulation/(8. * np.arctan(1.0))
        return circulation

In [3]:
class AMReX_CL_Data(CL_Analysis):
    def __init__(self, plotfile):
        ds = yt.load(plotfile)
        cg = ds.covering_grid(left_edge=ds.domain_left_edge, dims=ds.domain_dimensions, level=0)
        coords = cg.fcoords
        
        self.domain_dimensions = ds.domain_dimensions

        xs = coords[:,0]
        ys = coords[:,1]
        ts = coords[:,2]

        self.dx = np.max(xs[1:] - xs[:-1])
        self.dy = np.max(ys[1:] - ys[:-1])
        self.dt = np.max(ts[1:] - ts[:-1])

        self.phi_1_Re = cg["phi_1_Re"][:,:,:].d
        self.phi_1_Im = cg["phi_1_Im"][:,:,:].d
        self.phi_2_Re = cg["phi_2_Re"][:,:,:].d
        self.phi_2_Im = cg["phi_2_Im"][:,:,:].d

    def get_fields(self, i,j,k):
        return self.phi_1_Re[i,j,k], self.phi_1_Im[i,j,k], self.phi_2_Re[i,j,k], self.phi_2_Im[i,j,k]

In [4]:
class NRRB_CL_Data(CL_Analysis):
    def __init__(self, filename):
        # Read lattice save file
        f_in = open(filename, "r")

        # Read 2 header lines
        f_in.readline()
        f_in.readline()

        data_dict = {}

        xcoord = []
        ycoord = []
        tcoord = []
        
        # Read cell & field values from file
        for l in f_in:
            ls = l.strip().split()
            coords = ls[0][1:-1].split(',')
            
            x = int(coords[0])
            if not x in xcoord:
                xcoord.append(x)
                
            y = int(coords[1])
            if not y in ycoord:
                ycoord.append(y)
                
            t = int(coords[2])
            if not t in tcoord:
                tcoord.append(t)
                
            p1_Re = float(ls[1])
            p1_Im = float(ls[2])
            p2_Re = float(ls[3])
            p2_Im = float(ls[4])
            data_dict[(x,y,t)] = (p1_Re, p1_Im, p2_Re, p2_Im)

        f_in.close()
        self.data = data_dict
        self.domain_dimensions = [len(xcoord), len(ycoord), len(tcoord)]
      
    def get_fields(self, i, j, k):
        return self.data[(i, j, k)]

In [5]:
# The following lattice files were generated compiling the NRRB and AMReX-CL versions with USE_TEST_CONSTANT_RNG=TRUE
amrex_cl_filename = "plt00001"
nrrb_filename = "../../field_configs/v4_mu_-0.100_w_0.100_Nx_21_nL_1_field_config.txt"

In [6]:
amrex_cl = AMReX_CL_Data(amrex_cl_filename)
nrrb = NRRB_CL_Data(nrrb_filename)

yt : [INFO     ] 2020-02-06 12:38:44,016 Parameters: current_time              = 0.01
yt : [INFO     ] 2020-02-06 12:38:44,020 Parameters: domain_dimensions         = [21 21 80]
yt : [INFO     ] 2020-02-06 12:38:44,023 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-02-06 12:38:44,027 Parameters: domain_right_edge         = [21. 21. 80.]


In [7]:
amrex_circ = amrex_cl.Circulation(2)
print(amrex_circ)

adding (8, 8): 6.815422950445793e-07
adding (12, 8): 6.815422950445793e-07
adding (8, 9): 2.2717644720948726e-07
adding (12, 9): -6.815422950445793e-07
adding (8, 10): -2.2717644720948726e-07
adding (12, 10): -2.2717644720948726e-07
adding (8, 11): -6.815422950445793e-07
adding (12, 11): 2.2717644720948726e-07
adding (8, 12): 6.815422950445793e-07
adding (12, 12): 6.815422950445793e-07
adding (9, 12): 2.2717644720948726e-07
adding (9, 8): -6.815422950445793e-07
adding (10, 12): -2.2717644720948726e-07
adding (10, 8): -2.2717644720948726e-07
adding (11, 12): -6.815422950445793e-07
adding (11, 8): 2.2717644720948726e-07
0.0


In [8]:
nrrb_circ = nrrb.Circulation(2)
print(nrrb_circ)

adding (8, 8): 7.951309770604098e-07
adding (12, 8): 7.951309770604098e-07
adding (8, 9): 1.1358798057692354e-07
adding (12, 9): -7.951309770604098e-07
adding (8, 10): -1.1358798057692354e-07
adding (12, 10): -1.1358798057692354e-07
adding (8, 11): -7.951309770604098e-07
adding (12, 11): 1.1358798057692354e-07
adding (8, 12): 7.951309770604098e-07
adding (12, 12): 7.951309770604098e-07
adding (9, 12): 1.1358798057692354e-07
adding (9, 8): -7.951309770604098e-07
adding (10, 12): -1.1358798057692354e-07
adding (10, 8): -1.1358798057692354e-07
adding (11, 12): -7.951309770604098e-07
adding (11, 8): 1.1358798057692354e-07
0.0


In [9]:
print(amrex_circ - nrrb_circ)

0.0
